Step 1: Install/Import Necessary Libraries

In [1]:
!pip install -U transformers datasets scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.9 MB/s eta 0:00:00


In [2]:
import numpy as np
import torch
from datasets import load_dataset
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          Trainer, TrainingArguments, DataCollatorWithPadding)
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score


Load the IMDb Dataset
Here we load the IMDb dataset (a binary sentiment dataset with labels 0 and 1)

In [3]:
# Load the dataset from Hugging Face
dataset = load_dataset("imdb")
print(dataset)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


Preprocess and Tokenize the Dataset

We use the distilbert-base-uncased tokenizer and map it over the dataset.

In [4]:
# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Define a tokenization function
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

# Apply tokenization to the entire dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Create Training, Validation, and Test Splits

The IMDb dataset comes with a predefined test split. We'll split a small portion of the training data to serve as a validation set

In [5]:
# Split the original train set into training and validation sets (80% train, 20% validation)
split_datasets = tokenized_datasets["train"].train_test_split(test_size=0.2)
train_dataset = split_datasets["train"]
val_dataset = split_datasets["test"]
test_dataset = tokenized_datasets["test"]

print(f"Train dataset: {len(train_dataset)} samples")
print(f"Validation dataset: {len(val_dataset)} samples")
print(f"Test dataset: {len(test_dataset)} samples")

Train dataset: 20000 samples
Validation dataset: 5000 samples
Test dataset: 25000 samples


Prepare a Data Collator

We use a data collator that will dynamically pad our inputs to the maximum length in each batch

In [6]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


Load a Pre-trained DistilBERT Model for Sequence Classification

In [7]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Define the Evaluation Metrics

We use accuracy, weighted F1, precision, and recall as our metrics

In [8]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    return {"accuracy": accuracy, "f1": f1, "precision": precision, "recall": recall}

Set Up Training Arguments and Initialize the Trainer

We configure the training arguments (including learning rate, batch sizes, number of epochs, etc.) and then initialize the Trainer.

In [14]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",         # Evaluate at the end of each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=50,
    load_best_model_at_end=True,         # Save the best model based on evaluation
    metric_for_best_model="accuracy",
    save_strategy="epoch",
    # Early stopping parameters:
    save_total_limit=2,  # Only keep the best 2 checkpoints
    #early_stopping_patience=2,  # Stop after 2 epochs without improvement
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-14-2cb2b480f1ba>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Fine-tune the Model

Run the training loop

In [15]:
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.170700,0.213292,0.919600,0.919600,0.919601,0.919600


TrainOutput(global_step=1250, training_loss=0.22624320755004884, metrics={'train_runtime': 999.8344, 'train_samples_per_second': 20.003, 'train_steps_per_second': 1.25, 'total_flos': 2627312849274816.0, 'train_loss': 0.22624320755004884, 'epoch': 1.0})

Evaluate the Model on the Test Set

After training, evaluate the performance on the test set

In [16]:
test_results = trainer.evaluate(test_dataset)
print("Test set results:", test_results)


Test set results: {'eval_loss': 0.19689615070819855, 'eval_accuracy': 0.92612, 'eval_f1': 0.9261199970448, 'eval_precision': 0.9261200681792109, 'eval_recall': 0.92612, 'eval_runtime': 355.2586, 'eval_samples_per_second': 70.371, 'eval_steps_per_second': 4.4, 'epoch': 1.0}


Save the Fine-tuned Model

In [17]:
trainer.save_model("./sentiment_model")
